In [ ]:
import pandas as pd
import numpy as np
import polars as pl
from math import acos, pi, sqrt
import random
import rtsvg
rt = rtsvg.RACETrack()

In [ ]:
class VennDiagram(object):
    #
    #
    #
    def __init__(self, **kwargs):
        self.rt_self   = kwargs['rt_self']
        self.df        = kwargs['df']
        self.fld0      = kwargs['fld0']
        self.fld1      = kwargs['fld1']
        self.widget_id = kwargs['widget_id']
        if self.widget_id is None: self.widget_id = "vennDiagram_" + str(random.randint(0,65535))
        self.x_ins     = kwargs['x_ins']
        self.y_ins     = kwargs['y_ins']
        self.w         = kwargs['w']
        self.h         = kwargs['h']
        self.count_by  = kwargs['count_by'] # None = row-based (default) or field_name = column-based or __set__ for set-based

        if self.count_by is None:
            self.df0 = self.df.groupby(self.fld0).size().reset_index().rename({self.fld0:'__item__', 0:'__count__'},axis=1)
            self.df1 = self.df.groupby(self.fld1).size().reset_index().rename({self.fld1:'__item__', 0:'__count__'},axis=1)
        elif self.count_by == '__set__':
            self.df0 = pd.DataFrame({'__item__':list(set(self.df[self.fld0]))})
            self.df0['__count__'] = 1
            self.df1 = pd.DataFrame({'__item__':list(set(self.df[self.fld1]))})
            self.df1['__count__'] = 1
        else:
            self.df0 = self.df.groupby(self.fld0)[self.count_by].sum().reset_index().rename({self.fld0:'__item__',self.count_by:'__count__'},axis=1)
            self.df1 = self.df.groupby(self.fld1)[self.count_by].sum().reset_index().rename({self.fld1:'__item__',self.count_by:'__count__'},axis=1)


        self.df_joined = self.df0.set_index('__item__').join(self.df1.set_index('__item__'), how='outer', lsuffix='0', rsuffix='1').fillna(0.0).reset_index()
        self.df_joined['__diff__']      = self.df_joined['__count__0'] - self.df_joined['__count__1']
        self.df_joined['__diffabs__']   = self.df_joined['__diff__'].abs()
        self.df_joined['__max__']       = self.df_joined[['__count__0','__count__1']].max(axis=1)
        self.df_joined['__minus_max__'] = self.df_joined['__max__'] - self.df_joined['__diffabs__']

        # 0 Only Size, Intersection Size, 1 Only Size
        self.size_0            = sum(self.df_joined.query('__diff__ >  0')['__diffabs__'])
        self.size_intersection = sum(self.df_joined['__minus_max__'])
        self.size_1            = sum(self.df_joined.query('__diff__ <  0')['__diffabs__'])

        self.set0_color         = '#0000ff'
        self.intersection_color = '#00ff00'
        self.set1_color         = '#ffdf00'

        self.last_render = None

    #
    # __overlapArea__() - calculate the area of overlap between two circles (x, y, r)
    # - https://mathworld.wolfram.com/Circle-CircleIntersection.html
    #
    def __overlapArea__(self, c0, c1):
        d = self.rt_self.segmentLength((c0,c1))
        r, R = c0[2], c1[2]
        p0 = r**2 * acos((d**2 + r**2 - R**2)/(2*d*r))
        p1 = R**2 * acos((d**2 + R**2 - r**2)/(2*d*R))
        p2 = sqrt((-d + r + R)*(d + r - R)*(d - r + R)*(d + r + R))
        return p0 + p1 - p2

    #
    # renderSVG() - render the SVG
    #
    def renderSVG(self):
        svg = [f'<svg id="{self.widget_id}" x="0" y="0" width="{self.w}" height="{self.h}" xmlns="http://www.w3.org/2000/svg">']
        svg.append(f'<rect x="0" y="0" width="{self.w}" height="{self.h}" fill="{self.rt_self.co_mgr.getTVColor("background","default")}" />')

        r_single_circle = min(self.w - 2*self.x_ins, self.h - 2*self.y_ins)/2.0 # if it were just a single circle
        r_equal         = (self.w - 2*self.x_ins) / 3.5                         # if they were both equal-sized circle w/out overlap (assumes x_ins and y_ins are the same... and w and h are the same)

        if   self.size_0 == 0 and self.size_1 == 0 and self.size_intersection == 0:
            _color_ = self.rt_self.co_mgr.getTVColor('label','error')
            svg.append(f'<line x1="{self.x_ins}"          y1="{self.y_ins}" x2="{self.w - self.x_ins}" y2="{self.h - self.y_ins}" stroke="{_color_}" stroke-width="0.5" />')
            svg.append(f'<line x1="{self.w - self.x_ins}" y1="{self.y_ins}" x2="{self.x_ins}"          y2="{self.h - self.y_ins}" stroke="{_color_}" stroke-width="0.5" />')
        elif self.size_0 == 0 and self.size_1 == 0: # only the intersection is filled
            svg.append(f'<circle cx="{self.w/2}" cy="{self.h/2}" r="{r_single_circle}" fill="{self.intersection_color}" fill-opacity="0.5" stroke="{self.intersection_color}" />') # green
        elif self.size_0 >  0 and self.size_1 == 0: # only 0 is filled ... maybe intersection is partially filled
            if self.size_0 > self.size_intersection:
                svg.append(f'<circle cx="{self.w/2}" cy="{self.h/2}" r="{r_single_circle}" fill="{self.set0_color}"         fill-opacity="0.5" stroke="{self.set0_color}" />') # blue
            else:
                svg.append(f'<circle cx="{self.w/2}" cy="{self.h/2}" r="{r_single_circle}" fill="{self.intersection_color}" fill-opacity="0.5" stroke="{self.intersection_color}" />') # green
        elif self.size_0 == 0 and self.size_1 >  0: # only 1 is filled ... maybe intersection is partially filled
            if self.size_1 > self.size_intersection:
                svg.append(f'<circle cx="{self.w/2}" cy="{self.h/2}" r="{r_single_circle}" fill="{self.set1_color}"         fill-opacity="0.5" stroke="{self.set1_color}" />') # yellow
            else:
                svg.append(f'<circle cx="{self.w/2}" cy="{self.h/2}" r="{r_single_circle}" fill="{self.intersection_color}" fill-opacity="0.5" stroke="{self.intersection_color}" />') # green
        elif self.size_0 >  0 and self.size_1 >  0 and self.size_intersection == 0: # both are filled ... intersection is empty
            if   self.size_0 == self.size_1: # same sized circles
                svg.append(f'<circle cx="{self.x_ins + r_equal}"          cy="{self.y_ins + r_equal}"          r="{r_equal}" fill="{self.set0_color}" stroke="{self.set0_color}" fill-opacity="0.5" />')
                svg.append(f'<circle cx="{self.w - self.x_ins - r_equal}" cy="{self.h - self.y_ins - r_equal}" r="{r_equal}" fill="{self.set1_color}" stroke="{self.set1_color}" fill-opacity="0.5" />')
            elif self.size_0 >  self.size_1:
                pass
            else:
                pass
        else:
            pass


        svg.append('</svg>')
        self.last_render = ''.join(svg)
        return self.last_render

    #
    # _repr_svg_()
    #
    def _repr_svg_(self):
        if self.last_render is None: self.renderSVG()
        return self.last_render

#
# vennDiagram() - create a VennDiagram
#
def vennDiagram(rt_self,
                df,
                fld0,
                fld1,
                count_by  = None, # None = row-based (default) or field_name = column-based or __set__ for set-based
                widget_id = None,
                x_ins     = 8,
                y_ins     = 8,
                w         = 128,
                h         = 128):
        vd = VennDiagram(rt_self=rt_self, df=df, fld0=fld0, fld1=fld1, count_by=count_by, widget_id=widget_id, x_ins=x_ins, y_ins=y_ins, w=w, h=h)
        return vd

params  = {} # {'x_ins':16, 'y_ins':16, 'w':128, 'h':128}
_tiles_ = []
_tiles_.append(vennDiagram(rt, pd.DataFrame({'f0':[],'f1':[]}), 'f0', 'f1', **params))   # Empty
df = pd.DataFrame({'f0':'a a a b c'.split(),'f1':'w w j c c'.split(),'ct':[5,5,9,4,1]})
_tiles_.append(vennDiagram(rt, df, 'f0', 'f1', count_by='ct',               **params))
_tiles_.append(vennDiagram(rt, df, 'f0', 'f1', count_by=None,               **params))
_tiles_.append(vennDiagram(rt, df, 'f0', 'f1', count_by='__set__',          **params))
df = pd.DataFrame({'a':[1,None],'b':[None,2],'n':[5,10]})
_tiles_.append(vennDiagram(rt, df, 'a', 'b',                                **params)) # Disjoint sets
_tiles_.append(vennDiagram(rt, df, 'a', 'b', count_by='n',                  **params)) # Disjoint sets w/ count
df = pd.DataFrame({'a':['a','c',None,None],'b':[None,None,'b','c'],'n':[5,10,8,9]})
_tiles_.append(vennDiagram(rt, df, 'a', 'b',                                **params)) # Partial overlap
_tiles_.append(vennDiagram(rt, df, 'a', 'b', count_by='n',                  **params)) # Partial overlap w/ count
df = pd.DataFrame({'x':['a','b',None],'y':[None,None,'a'],'z':[2,4,5]})
_tiles_.append(vennDiagram(rt, df, 'x', 'y',                                **params)) # One inside the other
_tiles_.append(vennDiagram(rt, df, 'x', 'y', count_by='z',                  **params)) # One inside the other w/ count
_tiles_.append(vennDiagram(rt, df, 'y', 'x',                                **params)) # One inside the other (reversed inputs)
_tiles_.append(vennDiagram(rt, df, 'y', 'x', count_by='z',                  **params)) # One inside the other (reversed inputs) w/ count
#rt.table(_tiles_, per_row=6, spacer=10)

In [ ]:
rt.tile([rt.svgVennDiagram(5, 5, 0), rt.svgVennDiagram(5, 5, 1), rt.svgVennDiagram(5, 5, 2), rt.svgVennDiagram(5, 5, 3), rt.svgVennDiagram(5, 5, 4), rt.svgVennDiagram(5, 5, 5)])

In [ ]:
params = {}
rt.table([rt.svgVennDiagram(10,   0,  0, **params), rt.svgVennDiagram(10,  5,  0, **params), rt.svgVennDiagram( 0,  10,  0, **params), rt.svgVennDiagram( 5,  10,  0, **params),
          rt.svgVennDiagram( 6,  10,  0, **params), rt.svgVennDiagram( 7, 10,  0, **params), rt.svgVennDiagram( 8,  10,  0, **params), rt.svgVennDiagram( 9,  10,  0, **params),
          rt.svgVennDiagram(10,  10,  0, **params), rt.svgVennDiagram(10,  0,  0, **params), rt.svgVennDiagram(10,   5,  1, **params), rt.svgVennDiagram( 0,  10,  0, **params),
          rt.svgVennDiagram( 1,  10,  1, **params), rt.svgVennDiagram( 6, 10,  2, **params), rt.svgVennDiagram( 7,  10,  3, **params), rt.svgVennDiagram( 8,  10,  4, **params),
          rt.svgVennDiagram( 9,  10,  5, **params), rt.svgVennDiagram(10, 10,  6, **params), rt.svgVennDiagram( 5,  10,  5, **params), rt.svgVennDiagram( 6,  10,  6, **params),
          rt.svgVennDiagram( 7,  10,  7, **params), rt.svgVennDiagram( 8, 10,  8, **params), rt.svgVennDiagram( 9,  10,  9, **params), rt.svgVennDiagram(10,  10, 10, **params),
          rt.svgVennDiagram(10,   5,  5, **params), rt.svgVennDiagram(10,  6,  6, **params), rt.svgVennDiagram(10,   7,  7, **params), rt.svgVennDiagram(10,   8,  8, **params), 
          rt.svgVennDiagram(10,   9,  9, **params), rt.svgVennDiagram(10, 10, 10, **params),], spacer=10, per_row=10)